In [148]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../../scripts/")
from importlib import reload
import TweetToVect2 as TW
import UserToVect as UVect
# reload(TW)
pd.options.display.max_colwidth = 200

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import dash
import dash_core_components as dcc
import dash_html_components as html

In [47]:
userToVect = UVect.UserToVect()

In [97]:
users_df = pd.read_pickle('../../datasets/users_vectores.pkl')
users_partidos = pd.read_csv('../../datasets/user_partido.csv')
users_partidos.columns=['user_id', 'user_name', 'partido', 'partido_original']
def unificar_partido(row):
    row.partido = row.partido if pd.isna(row.partido_original) else row.partido_original
    return row
users_partidos = users_partidos.apply(unificar_partido,axis=1)
users_partidos = users_partidos[users_partidos.user_id.isin(users_df.user_id)]
users_df.loc[users_partidos.user_id.values, 'partido'] = users_partidos.partido.values
users_df.loc[users_partidos.user_id.values, 'partido_original'] = users_partidos.partido_original.values

In [158]:
partidos = ['JUNTOS POR EL CAMBIO', 'FRENTE DE TODOS', 'PTS - FRENTE DE IZQUIERDA']
users_todos = users_df
users_partidos_todos = users_df[users_df.partido.isin(partidos)]
users_partidos = users_df[users_df.partido.isin(partidos)]

In [159]:
sample_index = users_partidos[users_partidos.partido.isin(['JUNTOS POR EL CAMBIO', 'FRENTE DE TODOS'])].sample(200).user_id.values
sample_2  = users_partidos[users_partidos.partido.isin(['PTS - FRENTE DE IZQUIERDA'])].sample(100).user_id.values
users_partidos = users_partidos[
    (users_partidos.user_id.isin(sample_index)).values |
    (pd.notna(users_partidos.partido_original)) |
    (users_partidos.user_id.isin(sample_2)).values
]
len(users_partidos)

392

In [160]:
def mean_partido(df):
    emb = np.array([i for i in df.vector_user.values])
    emb = np.mean(np.transpose(emb),1)
    return emb

emb_partidos = users_partidos.groupby('partido').apply(mean_partido)
emb_partidos

partido
FRENTE DE TODOS              [0.32634136, -0.2972437, -0.23059939, -0.23453386, 0.15805261, -0.260078, 0.023805913, -0.17431358, 0.3502834, 0.026385019, 0.27645656, -0.32957304, -0.12984742, 0.049298532, -0.01995939, 0.102952...
JUNTOS POR EL CAMBIO         [0.28948292, -0.26130885, -0.21443357, -0.26832497, 0.25151613, -0.13887995, 0.27652967, -0.12822078, 0.46887362, 0.14339337, 0.36391503, -0.18396188, -0.02296993, 0.095439106, -0.02454133, -0.118...
PTS - FRENTE DE IZQUIERDA    [0.14552918, -0.39489, -0.36044356, -0.26648933, 0.20373218, -0.33253083, 0.014711975, -0.1696655, -0.1665581, -0.44335175, 0.13378207, -0.6001606, -0.013978509, -0.02118956, -0.16408563, -0.04631...
dtype: object

In [161]:
def similar_to_vector(user_vector, df):
    sim = userToVect.pairwise_similarity_vectors(
        [user_vector],
        [i for i in df.vector_user.values]
    )
    u_copy = df.copy()
    u_copy.loc[:,'sim'] = sim[0]
    u_copy = u_copy.sort_values('sim', ascending=False)
    return u_copy

def similar_to_user(user_name, df, include_in_result=False):
    user = df[df.user_name.str.lower() == user_name.lower()]
    if(len(user)==0):
        raise Exception('Usuario no encontrado :(')
    user_vector = user.vector_user.values[0]
    u_copy = similar_to_vector(user_vector, df)
    if(not include_in_result):
        u_copy = u_copy[u_copy.user_name.str.lower() != user_name.lower()]
    return u_copy
    
similar_to_user('Senadores_Todos', users_df).head(4)

,user_id,user_name,name,description,url,created_at,geo_enabled,verified,profile_image_url,profile_image_url_large,...,pinned_tweet,tweets_scraped,tweets_scraped_at,scrapped_at,ignore,vector_user,n_tweets,partido,partido_original,sim
3014877580,3014877580,Diputados_Todos,Diputados de TOD☀S,Bloque de diputados y diputadas nacionales del @FrenteDeTodos.,https://t.co/8LYTxhXVA4,2015-02-03 17:33:20.0,1,1,https://pbs.twimg.com/profile_images/1227303736210526210/uZnSoOdf_normal.jpg,https://pbs.twimg.com/profile_images/1227303736210526210/uZnSoOdf_400x400.jpg,...,NaN,0,NaN,2020-07-28 02:36:35.0,0,"[-0.123096816, -0.6750792, -0.43815613, -0.23821145, 0.4973364, -0.581591, 0.24589351, 0.013269221, 0.61322963, 0.1659354, 0.39847106, -0.0700951, 0.41364104, 0.099161915, -0.14366613, 0.0912362, ...",87,NaN,NaN,0.818344
1641872756,1641872756,AnaMariaIanni,Ana María Ianni,Senadora Nacional por Santa Cruz - Diputada Nacional mandato cumplido,NaN,2013-08-03 02:48:08.0,1,0,https://pbs.twimg.com/profile_images/940965275226333185/nNxFa5ZI_normal.jpg,https://pbs.twimg.com/profile_images/940965275226333185/nNxFa5ZI_400x400.jpg,...,NaN,1,2020-07-28 13:44:53.0,2020-07-28 02:16:32.0,0,"[0.17780736, -0.38436913, -0.23587579, -0.107254975, -0.016501946, -0.24053325, 0.25212148, 0.12581533, 0.6965131, 0.16576375, 0.5498194, -0.21997607, -0.12313138, 0.16181965, 0.1160208, 0.0272403...",330,FRENTE DE TODOS,NaN,0.797890
888107793000968195,888107793000968195,noragimenezok,Nora del Valle Gimenez,"Senadora Nacional por #Salta, #FrenteDeTodos",NaN,2017-07-20 18:46:12.0,0,0,https://pbs.twimg.com/profile_images/1229800963759919104/hSGYbZTM_normal.jpg,https://pbs.twimg.com/profile_images/1229800963759919104/hSGYbZTM_400x400.jpg,...,NaN,1,2020-07-28 03:15:16.0,2020-07-28 02:18:09.0,0,"[0.2565584, -0.27794096, -0.32256517, 0.122719206, 0.15242997, -0.49745926, 0.23935027, -0.18485112, 0.39705035, 0.29207638, 0.37972927, -0.2741896, -0.123292185, 0.22463404, 0.28506657, 0.2628977...",122,FRENTE DE TODOS,NaN,0.792956
555684585,555684585,SenadoArgentina,Senado Argentina,Cuenta oficial del Senado de la Nación Argentina,https://t.co/mXQKd5M7Fx,2012-04-17 02:23:27.0,1,1,https://pbs.twimg.com/profile_images/1277447899048218624/a_vfqeI1_normal.jpg,https://pbs.twimg.com/profile_images/1277447899048218624/a_vfqeI1_400x400.jpg,...,NaN,1,2020-07-28 11:20:05.0,2020-07-28 02:12:23.0,0,"[0.1841145, -0.7393883, -0.093774185, 0.03373855, 0.07500252, -0.35332954, 0.40697175, -0.22448276, 0.46897578, 0.033502582, 0.38804665, -0.5914762, 0.1302191, 0.3796904, 0.264769, 0.014013602, -0...",1826,NaN,NaN,0.788047


## cambiar de partido
Tomar un usuario de un partido, restarle el embedding de su partido y sumarle el de otro

In [162]:
user_to_change = 'Senadores_Todos'
user = users_df[users_df.user_name.str.lower() == user_to_change.lower()]
user_vector = user.vector_user.values[0]
vector_partido = emb_partidos['FRENTE DE TODOS']
vector_target = emb_partidos['JUNTOS POR EL CAMBIO']
vector_target_2 = emb_partidos['PTS - FRENTE DE IZQUIERDA']

lambda_change = 2
user_result = np.array(user_vector) - np.array(vector_partido)*lambda_change + np.array(vector_target)*lambda_change
user_result_2 = np.array(user_vector) - np.array(vector_partido)*lambda_change + np.array(vector_target_2)*lambda_change

print('Original FRENTE DE TODOS')
display(similar_to_vector(user_vector, users_partidos_todos)[['user_name', 'name', 'description', 'partido', 'sim']].head(10))
print('Despues de transformar a JUNTOS POR EL CAMBIO')
display(similar_to_vector(user_result, users_partidos_todos)[['user_name', 'name', 'description', 'partido', 'sim']].head(10))
print('Despues de transformar a PTS - FRENTE DE IZQUIERDA')
display(similar_to_vector(user_result_2, users_partidos_todos)[['user_name', 'name', 'description', 'partido', 'sim']].head(10))

Original FRENTE DE TODOS


,user_name,name,description,partido,sim
1199715393583144962,Senadores_Todos,Senadores de TOD☀️S,Bloque de Senadores y Senadoras del @FrenteDeTodos,FRENTE DE TODOS,1.000000
1641872756,AnaMariaIanni,Ana María Ianni,Senadora Nacional por Santa Cruz - Diputada Nacional mandato cumplido,FRENTE DE TODOS,0.797890
888107793000968195,noragimenezok,Nora del Valle Gimenez,"Senadora Nacional por #Salta, #FrenteDeTodos",FRENTE DE TODOS,0.792956
588761482,AnaAlmironCtes,Ana Almirón,Senadora Nacional por Corrientes. Militante del Frente de Tod☀️S y La Campora 🇦🇷 \nAbogada y defensora de los intereses de los correntinos y correntinas,FRENTE DE TODOS,0.777087
2353385744,OscarParrilli,Oscar Parrilli,#OscarParrillienLugares Senador por Neuquén @frentedetodos. Administrado por militantes de OP.,FRENTE DE TODOS,0.763912
3005444542,BeatrizMirkinOk,Beatriz Mirkin,Senadora Nacional por Tucumán junto a @jalperovichok. Peronista. Militante por los derechos de las mujeres./ Instagram: @beatrizmirkinok,FRENTE DE TODOS,0.749807
466887457,CaroGaillard,Carito Gaillard,Abogada. Entrerriana. Dip. Nac. por la Pcia. de ER en Frente de Todos. Presidenta de la Comisión de Legislacion Penal. HCDN. Mamá de Felipe 💚🌈,FRENTE DE TODOS,0.737006
111921382,vegaclara,Clarita Vega,Senadora Nacional -La Rioja - Lic Ciencias Políticas - Docente- Militante por los Derechos de la Mujer. https://t.co/cFBHt39n7d,FRENTE DE TODOS,0.725704
1005105132386897920,SenadoresFPVPJ,Senadores FPV-PJ,Cuenta oficial del Bloque de Senadores y Senadoras del FPV-PJ,FRENTE DE TODOS,0.723577
1225026336802189312,Senadora_Olalla,Senadora Stella Olalla,Senadora Nacional por Entre Ríos\nContacto: prensasenadoraolalla@gmail.com,JUNTOS POR EL CAMBIO,0.717857


Despues de transformar a JUNTOS POR EL CAMBIO


,user_name,name,description,partido,sim
1199715393583144962,Senadores_Todos,Senadores de TOD☀️S,Bloque de Senadores y Senadoras del @FrenteDeTodos,FRENTE DE TODOS,0.854770
968222575733235712,CambiemosSenado,Juntos por el Cambio Senado,Cuenta oficial del Interbloque @cambiemos en el @SenadoArgentina Facebook https://t.co/LjXqgzOJDK,JUNTOS POR EL CAMBIO,0.793892
1225026336802189312,Senadora_Olalla,Senadora Stella Olalla,Senadora Nacional por Entre Ríos\nContacto: prensasenadoraolalla@gmail.com,JUNTOS POR EL CAMBIO,0.788256
250297107,diputadosucr,Bloque UCR - HCDN,Secretaría de Prensa del Bloque de Diputados Nacionales de la UCR prensa.bucr@hcdn.gob.ar,JUNTOS POR EL CAMBIO,0.727079
1641872756,AnaMariaIanni,Ana María Ianni,Senadora Nacional por Santa Cruz - Diputada Nacional mandato cumplido,FRENTE DE TODOS,0.725726
111921382,vegaclara,Clarita Vega,Senadora Nacional -La Rioja - Lic Ciencias Políticas - Docente- Militante por los Derechos de la Mujer. https://t.co/cFBHt39n7d,FRENTE DE TODOS,0.718127
199939113,enriquezjorge,Jorge R. Enríquez,"Diputado Nacional @juntoscambioar 💪🇦🇷 por la Ciudad Autónoma de Buenos Aires. Abogado UBA, Colegio Nacional de Buenos Aires. Un porteño de Ley. #ProVida 💙",JUNTOS POR EL CAMBIO,0.717087
150684554,_alvarogonzalez,Álvaro González,Abogado. Diputado Nacional por #JuntosPorElCambio por la Ciudad de Buenos Aires. Vicepresidente Primero de @DiputadosAR 🇦🇷,JUNTOS POR EL CAMBIO,0.712224
710486602909401088,blancopabloda,Pablo Daniel Blanco,"Senador de la Nación por Tierra del Fuego, Antártida e Islas del Atlántico Sur. #JuntosPorElCambio. Mi compromiso es con todos los fueginos.",JUNTOS POR EL CAMBIO,0.708765
62584305,VMoralesGorleri,Victoria Morales Gorleri,Diputada Nacional por la Ciudad de Buenos Aires. Bloque PRO. Juntos por el Cambio.,JUNTOS POR EL CAMBIO,0.708435


Despues de transformar a PTS - FRENTE DE IZQUIERDA


,user_name,name,description,partido,sim
301464513,Fte_Izquierda,Frente de Izquierda,"#FITunidad 💚 Alianza entre @PTSarg, @PartidoObrero e @IzqSocialista.",PTS - FRENTE DE IZQUIERDA,0.799448
1644225854,EstecheJor,Jorgelina Esteche,Docente. Candidata a primer concejal en Morón FIT Unidad. Congresal Suteba Corriente Nac. Docente 9 de abril La Marrón (PTS +independientes) Pan y Rosas,PTS - FRENTE DE IZQUIERDA,0.792450
198579422,PTSarg,PTS,Partido de los Trabajadores Socialistas en el @Fte_Izquierda. Impulsamos @izquierdadiario. Miembro de la @FT_CI.,PTS - FRENTE DE IZQUIERDA,0.791864
2158974572,RaulGodoyPTS,Raúl Godoy,Obrero de Zanon. Dirigente del PTS. Diputado del Frente de Izquierda en Neuquén (MC),PTS - FRENTE DE IZQUIERDA,0.789666
200638297,chipicastillo,Christian Castillo,Dirigente Nacional del PTS en el Frente de Izquierda. Sociólogo y docente universitario (UBA y UNLP),PTS - FRENTE DE IZQUIERDA,0.779830
1883432797,CeciMancuso,Cecilia Mancuso,Militante de la @JuventudDelPTS. Impulsora de la agrupación de mujeres @PanyRosas_Arg en el @Fte_Izquierda.,PTS - FRENTE DE IZQUIERDA,0.778157
880151392731705346,MicaBlancoMinol,Micaela Blanco Minoli,Concejal de Lavalle por el @PTSarg @Fte_Izquierda\n| Feminista socialista | Militante de @PanyRosas_Arg,PTS - FRENTE DE IZQUIERDA,0.767978
604018650,PTSPrensa,Marcela Soler,Prensa del Partido de los Trabajadores Socialistas (@PTSarg) de Argentina // Frente de Izquierda y de los Trabajadores (@Fte_Izquierda),PTS - FRENTE DE IZQUIERDA,0.766832
636591284,chaplinzanon,Andrés Blanco,Obrero de Zanon. Diputado Provincial del @PTSarg @Fte_Izquierda,PTS - FRENTE DE IZQUIERDA,0.765898
3106689671,OrtegaLu_,Lucía Ortega,"Economía, fútbol, feminismo. Columnista de @izquierdadiario\n#SeTeníaQueDecir 10:30 hs- Con la voz de las mujeres: otra mirada de la realidad, para transformarla",PTS - FRENTE DE IZQUIERDA,0.765084


Otro usuario

In [219]:
users_todos[users_todos.user_name.str.match('MaxiKirchnerOK', False)==True]

,user_id,user_name,name,description,url,created_at,geo_enabled,verified,profile_image_url,profile_image_url_large,...,media_count,pinned_tweet,tweets_scraped,tweets_scraped_at,scrapped_at,ignore,vector_user,n_tweets,partido,partido_original
874311190549323778,874311190549323778,MaxiKirchnerOK,Maximo Kirchner,🇦🇷 Militante Peronista. Diputado Nacional por Buenos Aires. 👉(Cuenta Fans)👈 lean por Favor 👈,https://t.co/axCGFnw0cC,2017-06-12 17:03:26.0,0,0,https://pbs.twimg.com/profile_images/1252077971252355075/9HZlV97L_normal.jpg,https://pbs.twimg.com/profile_images/1252077971252355075/9HZlV97L_400x400.jpg,...,100,1.257129e+18,1,2020-07-28 16:50:52.0,2020-07-28 02:18:32.0,0,"[0.2119288, -0.19456214, -0.25721493, -0.47249666, 0.25343984, -0.6809192, 0.24908006, 0.13863207, 0.35394442, 0.08339983, 0.47551736, -0.09758102, -0.15545274, 0.27559572, -0.14989127, 0.05210795...",65,FRENTE DE TODOS,FRENTE DE TODOS


In [170]:
user_to_change = 'PRO_Mujeres'
user = users_df[users_df.user_name.str.lower() == user_to_change.lower()]
user_vector = user.vector_user.values[0]
vector_partido = emb_partidos['JUNTOS POR EL CAMBIO']
vector_target = emb_partidos['FRENTE DE TODOS']
vector_target_2 = emb_partidos['PTS - FRENTE DE IZQUIERDA']

lambda_change = 2
user_result = np.array(user_vector) - np.array(vector_partido)*lambda_change + np.array(vector_target)*lambda_change
user_result_2 = np.array(user_vector) - np.array(vector_partido)*lambda_change/2 + np.array(vector_target_2)*lambda_change/2

print('Original JUNTOS POR EL CAMBIO')
display(similar_to_vector(user_vector, users_partidos_todos)[['user_name', 'name', 'description', 'partido', 'sim']].head(10))
print('Despues de transformar a FRENTE DE TODOS')
display(similar_to_vector(user_result, users_partidos_todos)[['user_name', 'name', 'description', 'partido', 'sim']].head(10))
print('Despues de transformar a PTS - FRENTE DE IZQUIERDA')
display(similar_to_vector(user_result_2, users_partidos_todos)[['user_name', 'name', 'description', 'partido', 'sim']].head(10))

Original JUNTOS POR EL CAMBIO


,user_name,name,description,partido,sim
126418051,PRO_Mujeres,PRO Mujeres,"PRO Mujeres: más mujeres, mejor política. Buscamos incrementar la participación política de las mujeres con perspectiva de género ¡Seguinos! 💪🙋🏻‍♀️",JUNTOS POR EL CAMBIO,1.000000
79533471,Sofiabrambilla,Sofia Brambilla,Mamá de Jose y Clemen. Diputada Nacional por Corrientes. Secretaria Nacional de @PRO_mujeres https://t.co/SbK4PJOulQ,JUNTOS POR EL CAMBIO,0.921921
971923363038973952,MujeresProCba,Pro Mujeres Cordoba,Cuenta Oficial de Pro Mujeres Córdoba #MásMujeresMejorPolítica,JUNTOS POR EL CAMBIO,0.908716
879898203482705921,jpromujerescba,JPRO Mujeres Córdoba,"Somos un equipo de mujeres comprometidas con el cambio y la realidad social. Más mujeres, mejor política!🚺🇦🇷💛",JUNTOS POR EL CAMBIO,0.879195
794011310,CaroBaroneBA,• Caro Barone •,• Lic. en Gobierno y Relaciones Internacionales • Mg. en Políticas Públicas • 💚💜Directora General de Relaciones Federales y Productivas en GCBA #BAFederal,JUNTOS POR EL CAMBIO,0.834118
159916083,lidiasaya,Lidia Saya,Asesora en el Min de Salud de CABA. Madre de tres soles. Psicóloga. De Boca. Nací en Chaco,JUNTOS POR EL CAMBIO,0.821751
1006704308547616768,promujereser,Pro Mujeres Entre Ríos,"Somos un grupo de mujeres que intercambian experiencias, comentarios y trabajos para aumentar su participación en la política",JUNTOS POR EL CAMBIO,0.816643
472996218,camicrescimbeni,Camila Crescimbeni 🇦🇷,"30 años, bonaerense y militante. Hincha de Racing. Consejo Nacional @proargentina. Diputada Nacional @juntoscambioar 🇦🇷",JUNTOS POR EL CAMBIO,0.810236
997190298,noracebreiro,Nora Cebreiro,@PROMujeresColon\n@WDNArgentina #NodoCórdoba @MediadoraComunitaria\n#NuestraMiradaesIndispensable \n#EstaciónJuárezCelman,JUNTOS POR EL CAMBIO,0.799197
226300920,fabianatunez,Fabiana Tuñez,"EX- Directora Ejecutiva Instituto Nacional de las Mujeres Defensora de Derechos Humanos de Mujeres, Niñas y Niños a vivir una vida libre de violencia",JUNTOS POR EL CAMBIO,0.773099


Despues de transformar a FRENTE DE TODOS


,user_name,name,description,partido,sim
126418051,PRO_Mujeres,PRO Mujeres,"PRO Mujeres: más mujeres, mejor política. Buscamos incrementar la participación política de las mujeres con perspectiva de género ¡Seguinos! 💪🙋🏻‍♀️",JUNTOS POR EL CAMBIO,0.806752
311667501,Lucia_GI,Lucía García Itzigsohn 💚,Directora Provincial de Planificación y Gestión Comunicacional en @minmujerespba,FRENTE DE TODOS,0.795390
236458875,EliGAlcorta,Eli Gomez Alcorta,"Ministra de las Mujeres, Géneros y Diversidad. Abogada DDHH. Docente (UBA). Feminista.",FRENTE DE TODOS,0.784609
1207713682718969856,MinGenerosAR,"Ministerio de las Mujeres, Géneros y Diversidad","Cuenta oficial del Ministerio de las Mujeres, Géneros y Diversidad de la Nación.",FRENTE DE TODOS,0.767462
794011310,CaroBaroneBA,• Caro Barone •,• Lic. en Gobierno y Relaciones Internacionales • Mg. en Políticas Públicas • 💚💜Directora General de Relaciones Federales y Productivas en GCBA #BAFederal,JUNTOS POR EL CAMBIO,0.764585
79533471,Sofiabrambilla,Sofia Brambilla,Mamá de Jose y Clemen. Diputada Nacional por Corrientes. Secretaria Nacional de @PRO_mujeres https://t.co/SbK4PJOulQ,JUNTOS POR EL CAMBIO,0.762550
917491089027817474,queridasok,Queridas,"Género, igualdad, violencia. Ayuda, difusión y reflexión. Desde el periodismo y el arte.",FRENTE DE TODOS,0.761303
133898264,georgibitz,Georgina Bitz,Mamá de Francesco. Nací y vivo en #SanMartín. 30 años. Militante Peronista. Concejala 2017/2021. #Peronismo2020,FRENTE DE TODOS,0.754070
2795307903,MoniMacha,Mónica Macha,"Militante kirchnerista por un proyecto plurinacional, popular, democrático y feminista. Lic. en Psicología. Diputada Nacional por Frente de Todos ☀️",FRENTE DE TODOS,0.736236
971923363038973952,MujeresProCba,Pro Mujeres Cordoba,Cuenta Oficial de Pro Mujeres Córdoba #MásMujeresMejorPolítica,JUNTOS POR EL CAMBIO,0.733950


Despues de transformar a PTS - FRENTE DE IZQUIERDA


,user_name,name,description,partido,sim
315999581,PanyRosas_Arg,Pan y Rosas,"Agrupación de mujeres Pan y Rosas (@PTSarg/@Fte_Izquierda) + Independientes). Nuestra biografía, acá: https://t.co/qhkFfeQT1q\nMás info en @izquierdadiario",PTS - FRENTE DE IZQUIERDA,0.794784
1060183369105989634,VeroCarrizoPts,Vero Carrizo,Militante del PTS en el Frente de Izquierda-Rosario\nPan y Rosas 💜💚,PTS - FRENTE DE IZQUIERDA,0.792399
1357962648,Sol_Bajar,Sol Bajar,Trotskista. Editora de Género y Sexualidades de @izquierdadiario. De lunes a viernes a las 10.30 h en #SeTeníaQueDecir.,PTS - FRENTE DE IZQUIERDA,0.771876
760157581,violeta_foetra,La Violeta en FOETRA,"Agrupación antiburocratica, antipatronal y clasista, que lucha por construir un sindicato verdaderamente democrático y por la unidad de las filas obreras",PTS - FRENTE DE IZQUIERDA,0.771519
724291128695087104,GuadaFantin,Guadalupe Fantin,"Docente. Referente del PTS. Candidata a intendenta del Frente de Izquierda en Río Cuarto, Córdoba.",PTS - FRENTE DE IZQUIERDA,0.756244
930282349039964160,MAClasistas,Movimiento de Agrupaciones Clasistas,Impulsado por el PTS en el Frente de Izquierda junto a trabajadores independientes de más de 60 gremios en todo el país.,PTS - FRENTE DE IZQUIERDA,0.742619
1241432381984309252,LadyMatanza,Lady Matanza💚,Red de Precarizadxs Organizadxs. Hago hilos de notas de La Izquierda Diario. \n Juventud del PTS📍 UNLaM,PTS - FRENTE DE IZQUIERDA,0.740732
761409192083218432,bren_hamilton,Brenda Hamilton,"Consejera Directiva FFyL UBA \n@JuventudDelPTS\n🤓🎬💚Estudiante de historia, cinéfila y \n#FeministaSocialista de @PanyRosas_Arg \n\n✍🏼@izquierdadiario",PTS - FRENTE DE IZQUIERDA,0.737095
925953326776311808,FrancoV79,Franco Villalba,"Obrero del PTS. Mi mas ardiente deseo es que los trabajadores sepan quiénes son sus amigos y quienes sus enemigos. Todo lo demás, merece mi absoluto desprecio.",PTS - FRENTE DE IZQUIERDA,0.725040
2158974572,RaulGodoyPTS,Raúl Godoy,Obrero de Zanon. Dirigente del PTS. Diputado del Frente de Izquierda en Neuquén (MC),PTS - FRENTE DE IZQUIERDA,0.719603


La cuenta mantiene sus caracteristicas de hablar sobre feminismo en general pero transforma el partido correctamente

In [225]:
def get_cuenta_modificada(user_to_change, partido):
    user = users_df[users_df.user_name.str.lower() == user_to_change.lower()]
    user_vector = user.vector_user.values[0]
    
    otros_partidos = emb_partidos[emb_partidos.index!=partido]
    
    vector_partido = emb_partidos[partido]
    vector_target = otros_partidos.iloc[0]
    vector_target_2 = otros_partidos.iloc[1]

    lambda_change = 1
    user_result = np.array(user_vector) - np.array(vector_partido)*lambda_change + np.array(vector_target)*lambda_change
    user_result_2 = np.array(user_vector) - np.array(vector_partido)*lambda_change + np.array(vector_target_2)*lambda_change
    
    return user_vector, user_result, user_result_2

In [226]:
df_to_plot = users_partidos[~users_partidos.user_name.isin(['PRO_Mujeres', 'Senadores_Todos'])]
vectores = df_to_plot.vector_user.values.tolist() # saco esta cuenta para no duplicarla
print(len(vectores))

cuentas_a_modificar = [
    ['PRO_Mujeres', 'JUNTOS POR EL CAMBIO'],
    ['MaxiKirchnerOK', 'FRENTE DE TODOS']
]

vectores_mod = []

for i in cuentas_a_modificar:
    vectores_mod += list(get_cuenta_modificada(i[0], i[1])) #agrego los 3 vectores de usuario

vectores = vectores_mod + vectores
print(len(vectores))
distancias = userToVect.pairwise_distance_vectors(vectores, vectores)

392
398


In [227]:
from sklearn.manifold import MDS
dim = 3
mds = MDS(n_components=dim, max_iter=10000, eps=0.005, random_state=0,
                   dissimilarity="precomputed", n_jobs=-1)
Y = mds.fit_transform(distancias)
Y.shape

(398, 3)

In [ ]:
plotData = pd.concat(
    [df_to_plot, pd.DataFrame(data = Y[len(cuentas_a_modificar)*3:], index = df_to_plot.index, columns=['x','y','z'])],
    axis=1
)


traces = []

for i in range(len(cuentas_a_modificar)):
    
    cuenta_mod = Y[i*3:i*3+3][[1,0,2]]
    traces.append(go.Scatter3d(
        name=cuentas_a_modificar[i][0],
        x = np.transpose(cuenta_mod)[0],
        y = np.transpose(cuenta_mod)[1],
        z = np.transpose(cuenta_mod)[2]
    ))

#append tr
for cname in plotData.partido.unique():
    df = plotData[plotData.partido==cname]
    traces.append(go.Scatter3d(
        name=cname,
        x=df.x,
        y=df.y,
        z=df.z,
        hovertext = "<b style=\"max-width:200px\">"+df.name+"</b><br>"+
                    df.description+'<br>'+
                    df.partido,
        hoverinfo='text',
        mode='markers',
        marker={
            'size':8,
            'opacity':0.5
        }
    ))

fig = go.Figure(data=traces)
fig.update_layout(
#     autosize=False,
#     width=800,
    height=1050
)
app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
], style={'height': '100vh'})
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



I0806 15:55:20.203087 23364 dash.py:1626] Dash is running on http://127.0.0.1:8050/



I0806 15:55:20.233101 23364 dash.py:1628]  Warning: This is a development server. Do not use app.run_server


 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production W

I0806 15:55:20.282082 23364 dash.py:1631]  in production, use a production WSGI server like gunicorn instead.



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
